# Nautilus Pod Creation
This notebook will help you create a Pod on the Nautilus cluster

## Step 0: Prerequisites
* You have been added to a Nautilus namespace
* You have cloned down the SDSS2025 repository

## Step 1: Load the Template YAML File

We will be using a template of a Pod creation YAML to make this process easy:

```yaml
apiVersion: v1
kind: Pod
metadata:
  name: {{pod_name}}
spec:
  automountServiceAccountToken: false
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: topology.kubernetes.io/region
            operator: In
            values:
            - us-central 
  containers:
    - name: {{pod_name}}
      image: ubuntu:20.04
      command: ["sh", "-c", "echo 'Im a new pod' && sleep infinity"]
      resources:
        limits:
          memory: 12Gi
          cpu: 2
        requests:
          memory: 10Gi
          cpu: 2
```

We are creating a pod in the central u.s. region.

Let's load our template using a template engine called `jinja2`:

In [ ]:
from jinja2 import Template

# read in the template
with open('../yaml/pod_template.yml') as file_:
    template = Template(file_.read())

## Step 2: Render and Save the YAML File

Now, we can replace the name of the pod we are going to create, and save the YAML File. 

Be sure to use something like your initials in the name of the pod. Every namespace can only have a single pod with a given name.

In [ ]:
# replace None 
POD_NAME = input("Enter Pod Name: ")

pod_spec = template.render(pod_name=POD_NAME)

Let's view our rendered YAML:

In [ ]:
print(pod_spec)

Finally, let's save the YAML to disk:

In [ ]:
with open("./pod.yml", "w") as file:
    file.write(pod_spec)

## Step 3: Create the Pod

Now, we need to invoke KubeCTL with our saved YAML file to create the Pod:

In [ ]:
! kubectl create -f ./pod.yml

## Step 4: Verify Pod Creation

The output of Step 3 should have been something such as:
```
pod/PODNMAE created
```

Let's view all of the pods in our namespace and ensure we find ours:

In [ ]:
! kubectl get pods

We can also get more information using the `describe` action:

In [ ]:
! kubetl describe pod PODNAME

## Step 5: Using the Pod


We can now use the pod using the `exec` command. Let's run a simple "hello world" echo:

In [ ]:
! kubectl exec -it PODNAME -- echo 'hello world'

## Step 6: Deleting the Pod

Once we've finished using the pod, let's remove it:

In [ ]:
! kubectl delete pod PODNAME